# 🧠 Fake News Challenge – Core Notebook

## 1. Load & Inspect Dataset

## 2. Preprocess Text

## 3. Exploratory Analysis
- Word frequencies
- Article lengths
- Named entities

## 4. Visualization

## 5. Prompt Engineering
- Article selection
- Prompt design
- LLM response capture

## 6. Reflection
